In [ ]:
# script to test error where depository save adds properties it later can't read

In [1]:
import os

import rmgpy.data.kinetics
import rmgpy.reaction
import rmgpy.species
import rmgpy.chemkin
import rmgpy.kinetics
from collections import OrderedDict

In [2]:
ref_library_path = "/home/harris.se/rmg/RMG-database/input/kinetics/"
family = 'Disproportionation'
ref_database = rmgpy.data.kinetics.KineticsDatabase()
ref_database.load(
    ref_library_path,
    libraries=[],
    families=[family]
)
training_depo = ref_database.families[family].get_training_depository()
print(len(training_depo.entries), 'training reactions')


137 training reactions


In [3]:
# define the species in the example
r1 = rmgpy.species.Species(smiles='[O]O')
r2 = rmgpy.species.Species(smiles='C[CH2]')
p1 = rmgpy.species.Species(smiles='OO')
p2 = rmgpy.species.Species(smiles='C=C')

my_reaction = rmgpy.reaction.Reaction()
my_reaction.reactants = [r1, r2]
my_reaction.products = [p1, p2]
my_reaction.elementary_high_p = True
my_reaction.kinetics = rmgpy.kinetics.Arrhenius(
    A=(100.07,'cm^3/(mol*s)'),
    n=3.43775,
    Ea=(48.6525,'kJ/mol'),
    T0=(1,'K'),
    Tmin=(298,'K'),
    Tmax=(2500,'K'),
    comment="""Fitted to 50 data points; dA = *|/ 1.15458, dn = +|- 0.0188838, dEa = +|- 0.102765 kJ/mol"""
)
print(my_reaction)

my_entry = rmgpy.data.base.Entry(
    label=str(my_reaction),
    item=my_reaction,
    data=my_reaction.kinetics
)


[O]O + C[CH2] <=> OO + C=C


In [4]:
# add it to the training depo -- need to do it to the actual Disproportionation family
print(len(training_depo.entries))
training_depo.entries[1000] = my_entry
print(len(training_depo.entries))


137
138


In [5]:
# save the training depo
training_depo.save_dictionary(os.path.join(ref_library_path, 'families', family, 'training', 'dictionary.txt'))
training_depo.save(os.path.join(ref_library_path, 'families', family, 'training', 'reactions.py'))


In [6]:
# try to read it again

ref_library_path = "/home/harris.se/rmg/RMG-database/input/kinetics/"
family = 'Disproportionation'
ref_database = rmgpy.data.kinetics.KineticsDatabase()
ref_database.load(
    ref_library_path,
    libraries=[],
    families=[family]
)
training_depo = ref_database.families[family].get_training_depository()
print(len(training_depo.entries), 'training reactions')

ERROR:root:Error while reading database '/home/harris.se/rmg/RMG-database/input/kinetics/families/Disproportionation/training/reactions.py'.
ERROR:root:Error when loading reaction family '/home/harris.se/rmg/RMG-database/input/kinetics/families/Disproportionation'


TypeError: load_entry() got an unexpected keyword argument 'elementary_high_p'